<a href="https://colab.research.google.com/github/DevAgarwal2111/DevAgarwal2111/blob/main/mcq_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain_google_genai

In [ ]:
import os
key= os.environ["GOOGLE_API_KEY"] = "google_api_key"

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
!pip install PyPDF2


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
import PyPDF2

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0.3)

In [ ]:
RESPONSE_JSON = """
"1": {
"mcq": "multiple choice question",
"options": {
"a": "choice here",
"b": "choice here",
"c": "choice here",
"d": "choice here",
},

"correct":"correct answer",
},

"2": {
"mcq": "multiple choice question",
"options": {
"a": "choice here",
"b": "choice here",
"c": "choice here",
"d": "choice here",
},

"correct": "correct answer",
},
"3": {
"mcq": "multiple choice question",
"options": {
"a": "choice here",
"b": "choice here",
"c": "choice here",
"d": "choice here",
},

"correct": "correct answer",
},
"""

In [ ]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for students in {tone} tone and {difficulty} difficulty level 'Easy', 'Medium' , or 'Hard'.
Make sure the questions are not repeated and check all the questions to be conforming the text as well
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [ ]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","tone","difficulty","response_json"],
    template=TEMPLATE
)

In [ ]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["quiz"], template="Given the following quiz in JSON format:\n{quiz}\n\nReview and evaluate the quality of this quiz based on the provided text. Provide feedback on the clarity, accuracy, and relevance of the questions and answers.")

In [ ]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [ ]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","tone","difficulty","response_json"],output_variables=["quiz","review"],verbose=True)

In [ ]:
import json

In [ ]:
def mcq_generator(file, number, DIFFICULTY):
  with open(file, 'rb') as f:
    TEXT = f.read()

  response = generate_evaluate_chain({
      "text": TEXT,
      "number": int(number),
      "tone": "simple",
      "difficulty": DIFFICULTY,
      "response_json": RESPONSE_JSON
  })

  quiz = response.get("quiz", None)
  print("DEBUG quiz raw value:", repr(quiz))

  if not quiz:
    return "No quiz generated. The model returned nothing."

  if isinstance(quiz, str):
    try:
      data = json.loads(quiz)
    except json.JSONDecodeError:
      return f"Quiz:\n{quiz}"
    else:
      data = quiz

  md_output = ""

  for qid, qdata in data.items():
      md_output += f"### Q{qid}. {qdata['mcq']}\n\n"
      for opt_key, opt_val in qdata["options"].items():
          md_output += f"- **{opt_key.upper()}**: {opt_val}\n"
      md_output += f"\n **Correct Answer**: {qdata['correct'].upper()}\n\n---\n"
  return md_output


In [ ]:
import gradio as gr

In [ ]:
iface = gr.Interface(
    fn=mcq_generator,
    inputs=[
        gr.File(label = "Upload"),
        gr.Number(label = "Number of Questions"),
        gr.Dropdown(choices=["Easy", "Medium", "Hard"], label = "Difficulty"),
    ],
    outputs = [
        gr.Markdown(label = "Quiz")
    ],
    title="MCQ",
    description = "Upload your text file, select number of questions, and difficulty level to generate a quiz."
)
iface.launch()